In [1]:
import sys
sys.executable

'/home/mohammedi/workspace/tfhe-ubiris/venv/bin/python'

In [2]:
import time

import numpy as np
import concrete.numpy as cnp

In [3]:
INPUT_SHAPE = (2**4,)

In [4]:
# Used to force the compiler to use a 16-bit
SCALE_UP_MAX_VALUE = 2**16

In [5]:
INPUT_MAX_VALUE = 2**4

In [6]:
verbose = False

In [7]:
def f(x, y) -> int:
    # x = (SCALE_UP_MAX_VALUE - 1) - ((SCALE_UP_MAX_VALUE - 1) - x)
    # y = (SCALE_UP_MAX_VALUE - 1) - ((SCALE_UP_MAX_VALUE - 1) - y)

    return np.sum(x ^ y)

inputset = [
    (
        np.random.randint(0, INPUT_MAX_VALUE, size=INPUT_SHAPE, dtype=np.int64),
        np.random.randint(0, INPUT_MAX_VALUE, size=INPUT_SHAPE, dtype=np.int64),
    )
    for _ in range(100)
]
configuration = cnp.Configuration(
    enable_unsafe_features=True,
    use_insecure_key_cache=True,
    insecure_key_cache_location=".keys",
)

compiler = cnp.Compiler(f, {"x": "encrypted", "y": "encrypted"})
circuit = compiler.compile(inputset, configuration, verbose=verbose)

In [8]:
print(circuit)

%0 = x                          # EncryptedTensor<uint4, shape=(16,)>        ∈ [0, 15]
%1 = y                          # EncryptedTensor<uint4, shape=(16,)>        ∈ [0, 15]
%2 = bitwise_xor(%0, %1)        # EncryptedTensor<uint4, shape=(16,)>        ∈ [0, 15]
%3 = sum(%2)                    # EncryptedScalar<uint8>                     ∈ [66, 168]
return %3


In [ ]:
start = time.time()
circuit.keygen()
end = time.time()
print("keygen took", end - start, "seconds")

In [ ]:
print(circuit)

In [19]:
sample_x = np.random.randint(0, 2**8, size=INPUT_SHAPE, dtype=np.int64)
sample_y = np.random.randint(0, 2**8, size=INPUT_SHAPE, dtype=np.int64)
args = circuit.encrypt(sample_x, sample_y)

In [20]:
start = time.time()
output = circuit.run(args)
end = time.time()
print("execution took", end - start, "seconds")

execution took 1784.006066083908 seconds


In [21]:
fhe_result = circuit.decrypt(output)

In [22]:
clear_result = f(sample_x, sample_y)

In [23]:
assert fhe_result == clear_result

In [24]:
1784 / 60

29.733333333333334